* 참고 문헌   
    Class-Balanced Loss Based on Effective Number of Samples/Yin Cui,  Menglin Jia, Tsung-Yi Lin, Yang Song, Serge Belongie   
    Data Augmentation of Wearable Sensor Data for Parkinson's Disease Monitoring using Convolutional Neural Networks / Terry Taewoong Um, Franz Michael Josef Pfister, Daniel Pichler, Satoshi Endo, Muriel Lang, Sandra Hirche, Urban Fietzek, Dana Kulić

* 코드 참조   
    https://dacon.io/competitions/official/235689/codeshare/2388?page=3&dtype=recent / hahaha님   
    https://dacon.io/competitions/official/235689/codeshare/2396?page=3&dtype=recent / ghghdfd님   
    https://dacon.io/competitions/official/235689/codeshare/2394?page=3&dtype=recent / Jamm님   

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import pandas as pd
from scipy import fftpack
from numpy.fft import *
from numpy.random import seed
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from transforms3d.axangles import axangle2mat
import warnings
warnings.filterwarnings(action='ignore')

2021-10-08 10:58:05.713958: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


# Feature engineering
* acc와 gy에 에너지 값 추가
* 측정된 값에 시간대비 변화량 추가
* 센서값을 푸리에 변환
* standard scaling. mean, std는 train data에서 얻어진 것을 사용

In [150]:
# 시간 변화량 계산 함수
def jerk_signal(signal, dt=0.02): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])
    
# 푸리에 변환 함수    
def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

# feature engineering
def feature_engineering(data_):
    data = data_.copy()
    
    # 에너지 변수 추가
    data['acc_Energy']=(data['acc_x']**2+data['acc_y']**2+data['acc_z']**2)**(1/3)

    data['gy_Energy']=(data['gy_x']**2+data['gy_y']**2+data['gy_z']**2)**(1/3)

    data['gy_acc_Energy']=((data['gy_x']-data['acc_x'])**2+(data['gy_y']-data['acc_y'])**2+(data['gy_z']-data['acc_z'])**2)**(1/3)
    
    # 시간 변화량 변수 추가
    data_dt=[]
    for i in tqdm(data['id'].unique()):
        temp=data.loc[data['id']==i]
        for v in data.columns[2:]:
            values=jerk_signal(temp[v].values)
            values=np.insert(values,0,0)
            temp.loc[:,v+'_dt']=values
        data_dt.append(temp)
    data = pd.concat(data_dt)
    
    # 측정 값에 푸리에 변환 적용
    fft=[]
    for i in tqdm(data['id'].unique()):
        temp=data.loc[data['id']==i]
        for i in data.columns[2:8]:
            temp[i]=fourier_transform_one_signal(temp[i].values)
        fft.append(temp)
    data=pd.concat(fft)
    
    return data

In [151]:
# standard scaling
# scaler가 None이면 fit_transform 하고 scaler 반환
# scaler가 None이 아니면 transform 하고 scaler 반환 x
def scaling(data_, scaler=None):
    data = data_
    col = data.columns
    
    if scaler:
        data.iloc[:,2:]= scaler.transform(data.iloc[:,2:])
        data = pd.DataFrame(data = data,columns =col)
        
        return data
    
    else:
        scaler = StandardScaler()
    
        data.iloc[:,2:]= scaler.fit_transform(data.iloc[:,2:])
        data = pd.DataFrame(data = data,columns =col)
        
        return data, scaler

# Augmentation
* 데이터 불균형을 해결하기 위해 26번을 제외한 sample에 augmentation 적용 --> oversampling
* rolling : 시계열을 정해진 지점 부터 미룸(rolling)
* rotation : x,y,z 축을 섞음
* permutation : 데이터를 구간을 나눈 후 재배치

In [152]:
# data rolling
# 인풋의 2/3 만 적용
def rolling(data_):
    data = data_.copy()
    sampling = np.random.choice(data.shape[0], int(data.shape[0] * 2 / 3))
    for j in sampling:
        data[j] = np.roll(data[j], np.random.choice(data.shape[1]), axis=0)
    return data

# 3개의 센서 값을 rotation
def rotation(data_):
    data = data_.copy()
    axis = np.random.uniform(low=-1, high=1, size=data.shape[1])
    angle = np.random.uniform(low=-np.pi, high=np.pi)
    return np.matmul(data, axangle2mat(axis, angle))

# 시계열을 구간으로 나눈 후 재 조합
def permutation(data_, nPerm=4, mSL=10):
    data = data_.copy()
    data_new = np.zeros(data.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm + 1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(mSL, data.shape[0] - mSL, nPerm - 1))
        segs[-1] = data.shape[0]
        if np.min(segs[1:] - segs[0:-1]) > mSL:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        data_temp = data[segs[idx[ii]]:segs[idx[ii] + 1], :]
        data_new[pp:pp + len(data_temp), :] = data_temp
        pp += len(data_temp)
    return data_new


# augmentation
# input의 절반을 생성
def augmentation(data_, labels):
    data = data_.copy()
    
    # rotation
    print('rotation...')
    ro_aug = []
    ro_label = []
    ro_sampling = np.random.choice(data.shape[0]//600, int((data.shape[0]//600) * 1 / 4))
    for j in tqdm(ro_sampling):
        columns = data.iloc[:, 2:].columns
        temp_df = data.iloc[j*600:(j+1)*600, :2]
        # acc
        acc = rotation(np.array(data.iloc[j*600:(j+1)*600, 2:5]))
        # gy
        gy = rotation(np.array(data.iloc[j*600:(j+1)*600, 5:]))
        
        temp = np.concatenate([acc, gy], axis=1)
        temp = pd.DataFrame(temp, columns=columns, index=temp_df.index)
        temp = pd.concat([temp_df, temp], axis=1)
        ro_label.append(labels[j])
        ro_aug.append(temp)
        
    ro_aug = pd.concat(ro_aug)
    ro_feature = feature_engineering(ro_aug)
    ro_sc, _ = scaling(ro_feature)
    ro_series = ro_sc.iloc[:, 2:].to_numpy().reshape(-1, 600, 18)
    ro_series = rolling(ro_series)

    # permutation
    print('permutation...')
    per_aug = []
    per_label = []
    per_sampling = np.random.choice(data.shape[0]//600, int((data.shape[0]//600) * 1 / 4))
    for j in tqdm(per_sampling):
        columns = data.iloc[:, 2:].columns
        temp_df = data.iloc[j*600:(j+1)*600, :2]
        # acc
        acc = permutation(np.array(data.iloc[j*600:(j+1)*600, 2:5]))
        # gy
        gy = permutation(np.array(data.iloc[j*600:(j+1)*600, 5:]))
        
        temp = np.concatenate([acc, gy], axis=1)
        temp = pd.DataFrame(temp, columns=columns, index=temp_df.index)
        temp = pd.concat([temp_df, temp], axis=1)
        per_label.append(labels[j])
        per_aug.append(temp)
    
    per_aug = pd.concat(per_aug)
    per_feature = feature_engineering(per_aug)
    per_sc, _ = scaling(per_feature)
    per_series = per_sc.iloc[:, 2:].to_numpy().reshape(-1, 600, 18)
    per_series = rolling(per_series)

    
    final = np.concatenate([ro_series, per_series], axis=0)
    final_label = np.concatenate([ro_label, per_label], axis=0)
    
    return final, final_label

# Training
* Batch size = 64
* augmentation 후 섞기 위해 data set을 만들 때 shuffle 추가
* loss는 weighted CE loss 사용
* K-fold를 사용

In [105]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
    cat_y = tf.keras.utils.to_categorical(labels)

    BATCH_SIZE = 64
    train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
    train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000, seed=42)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return train_dataset

def make_val(series_data, labels):
    cat_y = tf.keras.utils.to_categorical(labels)

    BATCH_SIZE = 64
    val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return val_dataset

In [153]:
# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Input([600, 18]),
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.1),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [154]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'aug.hdf5'
checkpoint_dir_path = os.path.join('checkpoint')
checkpoint_path = os.path.join('checkpoint', ckpt_name)

# check checkpoint paht
# 경로가 없으면 생성함
if not(os.path.exists(checkpoint_dir_path)):
    os.mkdir(checkpoint_dir_path)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    ),
    # 4번 동안 val_loss의 향상이 없으면 lr 절반 감소
    tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5)
]

In [155]:
# loss에 적용할 weight를 만드는 함수
# sample 수가 적을 수록 최대 2배의 loss 적용
def make_weight(labels):
    beta=0.9
    temp = np.unique(labels, return_counts=True)
    weight = (1.0 - beta) / (1.0-np.power(beta, temp[1]))
    weight = MinMaxScaler((1,2)).fit_transform(weight.reshape(-1, 1))
    weight = {i: c[0] for i, c in enumerate(weight)}

    return weight

In [133]:
# data load
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [143]:
# id  획득을 위한 데이터 생성
series_data = train.to_numpy().reshape(-1, 600, 8)

In [144]:
k = 10 

split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []       # 매 fold 훈련 후 해당 모델의 accuracy 저장

for i, (train_id, val_id) in enumerate(split.split(series_data, labels)):
    series_train, labels_train = series_data[train_id], labels[train_id]
    series_val, labels_val = series_data[val_id], labels[val_id]
    
    # train data sample 수 기준으로 weight 계산
    weight = make_weight(labels_train)
    
    # validation
    val_data = pd.DataFrame(series_val.reshape(series_val.shape[0]*600, -1), columns=train.columns)
    val_data = feature_engineering(val_data)
    val_data, _ = scaling(val_data)
    series_val = val_data.iloc[:, 2:].to_numpy().reshape(-1, 600, 18)
   
    # oversampling
    # 26 번에 대해서는 적용 x
    aug_id = np.where(labels_train != 26)
    series_aug = series_train[aug_id]
    labels_aug = labels_train[aug_id]
    
    aug_data = pd.DataFrame(series_aug.reshape(series_aug.shape[0]*600, -1), columns=train.columns)
    series_aug, labels_aug = augmentation(aug_data, labels_aug)
    
    train_data = pd.DataFrame(series_train.reshape(series_train.shape[0]*600, -1), columns=train.columns)
    train_data = feature_engineering(train_data)
    train_data, _ = scaling(train_data)
    series_train = train_data.iloc[:, 2:].to_numpy().reshape(-1, 600, 18)
    
    series_train = np.concatenate([series_train, series_aug], axis=0)
    labels_train = np.concatenate([labels_train, labels_aug], axis=0)
    
    
    train_dataset = make_train(series_train, labels_train)
    val_dataset = make_val(series_val, labels_val)
    model = base()
    
    # class_weight : CE에 Weight 를 곱함
    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, 
              class_weight=weight, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

100%|████████████████████████████████████████| 313/313 [00:00<00:00, 970.30it/s]


rotation...


100%|███████████████████████████████████████| 324/324 [00:00<00:00, 1002.50it/s]


permutation...


100%|██████████████████████████████████████| 2812/2812 [00:04<00:00, 582.64it/s]


...

rotation...


100%|████████████████████████████████████████| 324/324 [00:00<00:00, 721.10it/s]


permutation...


100%|██████████████████████████████████████| 2813/2813 [00:04<00:00, 600.52it/s]


Epoch 1/100
56/56 [==============================] - 1s 20ms/step - loss: 3.6400 - accuracy: 0.3796 - val_loss: 3.3326 - val_accuracy: 0.2917
Epoch 2/100
56/56 [==============================] - 1s 18ms/step - loss: 3.0417 - accuracy: 0.4467 - val_loss: 2.0900 - val_accuracy: 0.5609
Epoch 3/100
56/56 [==============================] - 1s 18ms/step - loss: 2.8055 - accuracy: 0.4750 - val_loss: 1.8455 - val_accuracy: 0.5417
Epoch 4/100
56/56 [==============================] - 1s 18ms/step - loss: 2.6024 - accuracy: 0.5044 - val_loss: 1.6771 - val_accuracy: 0.6090
Epoch 5/100
56/56 [==============================] - 1s 18ms/step - loss: 2.4095 - accuracy: 0.5270 - val_loss: 1.8449 - val_accuracy: 0.5481
Epoch 6/100
56/56 [==============================] - 1s 18ms/step - loss: 2.2626 - accuracy: 0.5528 - val_loss: 1.4803 - val_accuracy: 0.6603
Epoch 7/100
56/56 [==============================] - 1s 18ms/step - loss: 2.1605 - accuracy: 0.5689 - val_loss: 1.6313 - val_accuracy: 0.6026
Epoch 

# Inference
* 각 fold 모델의 결과 값을 평균하여 사용

In [147]:
series_test = feature_engineering(test)
series_test = scaling(series_test, scaler)
series_test = series_test.iloc[:, 2:].to_numpy().reshape(-1, 600, 18)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 792.92it/s]


In [148]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [149]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/final_wt_aug.csv')     # 저장 경로
sub.to_csv(save_path, index=False)